In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ignore warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
X_train = pd.read_csv('/kaggle/input/santander-customer-satisfaction/train.csv')

X_test = pd.read_csv('/kaggle/input/santander-customer-satisfaction/test.csv')

In [ ]:
target = X_train['TARGET']

In [ ]:
target.value_counts()

In [ ]:
# 3008/73012
print((3008/(73012+3008))*100)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
%matplotlib inline
plt.figure(dpi=100)
sns.set_style('darkgrid')
sns.countplot(target, data=target)
plt.xlabel('target')
plt.ylabel('Count')
plt.xticks([0,1],['Satisfied','Not Satisfied'])
plt.show()

In [ ]:
# drop TARGET label from X_train

X_train.drop(labels=['TARGET'], axis=1, inplace = True)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
#Checking Missing Values
X_train.isnull().sum().any()

In [ ]:
#using sklearn variancethreshold to find constant features

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.01)
sel.fit(X_train)  # fit finds the features with zero variance

In [ ]:
# get_support is a boolean vector that indicates which features are retained
# if we sum over get_support, we get the number of features that are not constant
sum(sel.get_support())

In [ ]:
# Lets Find non-constant features 
len(X_train.columns[sel.get_support()])

In [ ]:
constant_columns = [column for column in X_train.columns
                    if column not in X_train.columns[sel.get_support()]]

print(len(constant_columns))

In [ ]:
for column in constant_columns:
    print(column)

In [ ]:
constant_columns

In [ ]:
X_train[['ind_var1','ind_var2']]

In [ ]:
X_train = X_train.drop(constant_columns,axis=1)

In [ ]:
X_train.shape

In [ ]:
# X_train['ind_var1']

In [ ]:
# 2: Feature Selection- With Correlation

import matplotlib.pyplot as plt
import seaborn as sns

#Using Pearson Correlation
corrmat = X_train.corr()
fig, ax = plt.subplots()
fig.set_size_inches(12,12)
sns.heatmap(corrmat,cmap="CMRmap_r")

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train, 0.98)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
 # DO NOT RUN THIS AS IT TAKES A LOT OF TIME 
    # 3 :-  Checking multicoliearity 

# # Import library for VIF
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# def calc_vif(X):

#     # Calculating VIF
#     vif = pd.DataFrame()
#     vif["variables"] = X.columns
#     vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#     return(vif)

In [ ]:
# it calculates VIF (Variable Inflation Factors).
# calc_vif(X_train)

In [ ]:
X_train = X_train.drop(corr_features,axis=1)

In [ ]:
X_train.shape

In [ ]:
# sns.heatmap(X_train.corr());

corrmat = X_train.corr()
fig, ax = plt.subplots()
fig.set_size_inches(12,12)
sns.heatmap(corrmat,cmap="CMRmap_r")

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

# X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [ ]:
X_train.head()

In [ ]:
print((X_train.shape,target.shape))

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(X_train,target,test_size=0.80,random_state=0,stratify=target)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)

print('Logistic Regression accuracy score with all the features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix (y_train,lrm.predict(x_train))


****Since the data is imballenced, we are now using SMOTE technique

In [ ]:
# from imblearn.combine import SMOTETomek
# smk=SMOTETomek(ratio=1,random_state=0)
# x_new,y_new=smk.fit_sample(X_train,y_train)
...
# transform the dataset

from imblearn.over_sampling import SMOTE
# print(imblearn.__version__)
oversample = SMOTE()
X, y = oversample.fit_resample(X_train, target)

In [ ]:
print(X.shape,y.shape)

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(X,y,test_size=0.80,random_state=0,stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression
lrm=LogisticRegression(C=0.1,penalty='l2',n_jobs=-1)
lrm.fit(x_train,y_train)

In [ ]:
y_pred=lrm.predict(x_test)
y_pred

In [ ]:
# y_pred = pd.DataFrame(y_pred)

In [ ]:
# y_train.shape, y_pred.shape

In [ ]:
from sklearn import metrics
print("Train Set Accuracy is ==> ",metrics.accuracy_score(y_train,lrm.predict(x_train)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix (y_train,lrm.predict(x_train))


Logistic Regression : Lets check how our model works if web are taking all columns

PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

In [ ]:
# plot the Cumulative Summation of the Explained Variance for the different number of components
# plt.figure()
# plt.plot(np.cumsum(X_train.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)') #for each component
# plt.title('Dataset Explained Variance')
# plt.show()

In [ ]:
#  4:  PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
pca_resampled_test = PCA().fit(X_train)

# plot the Cumulative Summation of the Explained Variance for the different number of components
plt.figure()
plt.plot(np.cumsum(pca_resampled_test.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.show()

In [ ]:
# instantiate PCA
pca = PCA(n_components=150)

# fit PCA
principalComponents = pca.fit_transform(X_train)

In [ ]:
train_pc = pd.DataFrame(data = principalComponents)
train_target = pd.Series(target, name='TARGET')

train_pc_df = pd.concat([train_pc, train_target], axis=1)
train_pc_df.head(5)

In [ ]:
sns.heatmap(train_pc.corr());

In [ ]:
# we calculate the variance explained by priciple component
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

Logistic regression after applying PCA

In [ ]:
train_pc_df.shape , train_target.shape

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(train_pc_df,train_target,test_size=0.80,random_state=0,stratify=train_target)

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)

print('Logistic Regression accuracy score with the first 150 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix (y_test, y_pred)   